### Question

*You have 26 black cards and 26 red cards. When you draw a red card, you lose one dollar. When you draw a black card, you win one dollar. You can always choose to end the game. What's the expected value of your strategy?*

The key is to find a recursive expression of the expected value. We can define the state in terms of B and R: $(B, R)$ where B is the number of black cards and R is the number of red cards. Given $(B, R)$, the next possible states is $(B-1, R)$ and $(B, R-1)$.

The best strategy should be "play the game if the expected value is bigger than 0, and don't otherwise." During the interview, my initial guess for the expected value was $E(B, R) = P(B) \cdot (1 + max(0, E(B-1, R))) + P(R) \cdot (-1 + max(0, E(B, R-1))$. However, there are some conditions that violate the strategy. For example, $E(1, 30) = \frac {1} {31} \cdot (1 + max(0, E(0, 30))) + \frac {30} {31} \cdot (-1 + max(0, E(1, 29)) = \frac {1} {31} + \frac {30} {31} \cdot (-1 + max(0, E(1, 29))$, and we can easily guess that the value would be negative. However, given that we can always end the game, it's not wise to keep playing the game if the expected value is negative, so the expected value should be 0 according to our strategy.

The modified expression would be $E(B, R) = max(0, P(B) \cdot (1 + E(B-1, R)) + P(R) \cdot (-1 + E(B, R-1)))$.

Using a naive recursion to implement it in code will result in the time complexity that is exponential to the number of cards. So, we will use dynamic programming to efficiently calculate the expected value of every possible state in advance in $O(B*R)$.

In [23]:
from collections import defaultdict
from tabulate import tabulate

In [45]:
class CardGame:
    def __init__(self, B, R):
        """
        :param B: number of total black cards
        :param R: number of total red cards
        """
        self.B = B
        self.R = R
        self.expected_values = defaultdict(lambda: defaultdict(lambda:-1))
        self.fill_up_expected_values(B, R)
        
    def fill_up_expected_values(self, B, R):
        """
        Given the maximum number of each color,
        calculate the expected value of every possible state in advance
        """
        for b in range(B+1):
            for r in range(R+1):
                self.expected_values[b][r] = self.get_expected_value(b, r)
        
    def get_card_probabilities(self, B, R):
        """
        :return: Probabilities of drawing B and R
        """
        num_cards = B + R
        prob_B = B / num_cards
        prob_R = R / num_cards
        return prob_B, prob_R
    
    def get_expected_value(self, B, R):
        """
        :return: the expected value of B and R
        """
        if B == 0:
            return 0
        if R == 0:
            return B
        
        prob_B, prob_R = self.get_card_probabilities(B, R)
        expected_value = max(0, prob_B * (1 + self.expected_values[B-1][R]) + prob_R * (-1 + self.expected_values[B][R-1]))
        return expected_value
    
    def print_expected_values(self):
        """
        Prints the expected values of all states in 2d table
        y axis: B
        x axis: R
        """
        matrix = []

        for b in range(B+1, -1, -1):
            values = list(dic[b].values())
            matrix += (["%.1f" % v for v in values]),

        print(tabulate(matrix))
    
    def solve(self, B, R):
        assert 0 < B and B <= self.B, "The number of black cards should be between 0 and " + str(self.B)
        assert 0 < R and R <= self.R, "The number of red cards should be between 0 and " + str(self.R)
        return self.expected_values[B][R]

In [46]:
dic = card_game.expected_values

In [47]:
B = 20
R = 20
card_game = CardGame(B, R)

In [48]:
card_game.solve(20, 4)

16.1948051948052

In [49]:
card_game.solve(40, 23)

AssertionError: The number of black cards should be between 0 and 20

In [50]:
card_game.print_expected_values()

--  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---

20  19    18.1  17.1  16.2  15.2  14.3  13.4  12.4  11.5  10.6  9.7  8.8  7.9  7    6.2  5.3  4.5  3.7  3    2.3
19  18.1  17.1  16.2  15.2  14.3  13.3  12.4  11.5  10.5   9.6  8.7  7.8  7    6.1  5.3  4.5  3.7  2.9  2.2  1.6
18  17.1  16.1  15.2  14.2  13.3  12.3  11.4  10.5   9.6   8.7  7.8  6.9  6    5.2  4.4  3.6  2.9  2.2  1.5  1
17  16.1  15.1  14.2  13.2  12.3  11.4  10.4   9.5   8.6   7.7  6.8  6    5.1  4.3  3.5  2.8  2.1  1.5  0.9  0.5
16  15.1  14.1  13.2  12.2  11.3  10.4   9.5   8.6   7.7   6.8  5.9  5.1  4.3  3.5  2.7  2    1.4  0.9  0.4  0.1
15  14.1  13.1  12.2  11.3  10.3   9.4   8.5   7.6   6.7   5.9  5    4.2  3.4  2.7  2    1.4  0.8  0.4  0.1  0
14  13.1  12.1  11.2  10.3   9.4   8.5   7.6   6.7   5.8   5    4.1  3.3  2.6  1.9  1.3  0.8  0.3  0.1  0    0
13  12.1  11.1  10.2   9.3   8.4   7.5   6.6   5.7   4.9   4.1  3.3  2.5  1.8  1.2  0.7  0.3  0    0 